<a href="https://colab.research.google.com/github/philippe753/CNN_project/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import glob
from matplotlib import pyplot as plt
import gc
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, AveragePooling2D, Conv2D, MaxPooling2D
from keras.preprocessing import image
from keras.optimizers import SGD, Adam
import cv2

In [ ]:
# unzip celebraties images
!unzip "/content/drive/MyDrive/AI_dataset/img_align_celeba.zip"

In [ ]:
# Importing data

data_Y = pd.read_excel("/content/drive/MyDrive/AI_dataset/list_attr_celeba.xlsx")
data_Y_columns = data_Y.iloc[0] # This contains the name of the features of the dataframe.

In [ ]:
# Inspecting Data

# plot of the first 5 columns of the raw data
print(data_Y.head(5))

# Deleating the first row, since it contains it just a interger refereing to the total number of imgs.
first_row = data_Y.iloc[0] #grab the first row for the header
data_Y = data_Y[1:] #take the data less the header row
data_Y.columns = first_row #set the header row as the df header
print('New header:')
data_Y.head(5)

       202599        Unnamed: 1  ...      Unnamed: 39 Unnamed: 40
0           #  5_o_Clock_Shadow  ...  Wearing_Necktie       Young
1  000001.jpg                -1  ...               -1           1
2  000002.jpg                -1  ...               -1           1
3  000003.jpg                -1  ...               -1           1
4  000004.jpg                -1  ...               -1           1

[5 rows x 41 columns]
New header:
0           # 5_o_Clock_Shadow  ... Wearing_Necktie Young
1  000001.jpg               -1  ...              -1     1
2  000002.jpg               -1  ...              -1     1
3  000003.jpg               -1  ...              -1     1
4  000004.jpg               -1  ...              -1     1
5  000005.jpg               -1  ...              -1     1

[5 rows x 41 columns]


## Randomly pick m images to be the training set.
Further, every image is flattened from a 3 dimmensional array to a 1-dimensional array.

In [ ]:
m = 2000  # number of examples to train the models
train_X = np.zeros((116412, m))
train_Y = pd.DataFrame(data=data_Y_columns, columns=data_Y_columns)
rand_num = np.random.randint(0,200000, m)  # random numbers to select the training set.
img_path = np.sort(glob.glob("img_align_celeba/*.*"))

for i in range(0, m):
    img = cv2.imread(img_path[rand_num[i]])
    # flat image into an array.
    img_v = img.reshape((img.shape[0] * img.shape[1] * img.shape[2], 1))
    train_X[:, i] = img_v[:, 0]
    # find the equivalent samples in data_Y.
    train_Y = pd.concat([train_Y, data_Y[data_Y['#'] == data_Y.iloc[rand_num[i]][0]]], ignore_index=True)


In [ ]:
# Visualizing the training set.


In [ ]:
# Implementing a 1 neural neural network (perceptron)

In [ ]:
# Implementing a multilayer deep neural neural network (perceptron)